# How to route between sub-chains

:::info Prerequisites

This guide assumes familiarity with the following concepts:
- [LangChain Expression Language (LCEL)](/docs/concepts/#langchain-expression-language)
- [Chaining runnables](/docs/how_to/sequence/)
- [Configuring chain parameters at runtime](/docs/how_to/configure)
- [Prompt templates](/docs/concepts/#prompt-templates)
- [Chat Messages](/docs/concepts/#message-types)

:::

Routing allows you to create non-deterministic chains where the output of a previous step defines the next step. Routing can help provide structure and consistency around interactions with models by allowing you to define states and use information related to those states as context to model calls.

There are two ways to perform routing:

1. Conditionally return runnables from a [`RunnableLambda`](/docs/how_to/functions) (recommended)
2. Using a `RunnableBranch` (legacy)

We'll illustrate both methods using a two step sequence where the first step classifies an input question as being about `LangChain`, `Anthropic`, or `Other`, then routes to a corresponding prompt chain.

## Example Setup
First, let's create a chain that will identify incoming questions as being about `LangChain`, `Anthropic`, or `Other`:

In [1]:
from langchain_anthropic import ChatAnthropic
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate

chain = (
    PromptTemplate.from_template(
        """Given the user question below, classify it as either being about `LangChain`, `Anthropic`, or `Other`.

Do not respond with more than one word.

<question>
{question}
</question>

Classification:"""
    )
    | ChatAnthropic(model_name="claude-3-haiku-20240307")
    | StrOutputParser()
)

chain.invoke({"question": "how do I call Anthropic?"})

'Anthropic'

Now, let's create three sub chains:

In [2]:
langchain_chain = PromptTemplate.from_template(
    """You are an expert in langchain. \
Always answer questions starting with "As Harrison Chase told me". \
Respond to the following question:

Question: {question}
Answer:"""
) | ChatAnthropic(model_name="claude-3-haiku-20240307")
anthropic_chain = PromptTemplate.from_template(
    """You are an expert in anthropic. \
Always answer questions starting with "As Dario Amodei told me". \
Respond to the following question:

Question: {question}
Answer:"""
) | ChatAnthropic(model_name="claude-3-haiku-20240307")
general_chain = PromptTemplate.from_template(
    """Respond to the following question:

Question: {question}
Answer:"""
) | ChatAnthropic(model_name="claude-3-haiku-20240307")

## Using a custom function (Recommended)

You can also use a custom function to route between different outputs. Here's an example:

In [3]:
def route(info):
    if "anthropic" in info["topic"].lower():
        return anthropic_chain
    elif "langchain" in info["topic"].lower():
        return langchain_chain
    else:
        return general_chain

In [4]:
from langchain_core.runnables import RunnableLambda

full_chain = {"topic": chain, "question": lambda x: x["question"]} | RunnableLambda(
    route
)

In [5]:
full_chain.invoke({"question": "how do I use Anthropic?"})

AIMessage(content="As Dario Amodei told me, to use Anthropic, you first need to understand the company's mission and approach. Anthropic is focused on developing safe and ethical artificial intelligence systems that can benefit humanity. \n\nTo use Anthropic's services, you can start by visiting their website at anthropic.com. There, you'll find information about their various AI products and services, including language models, data annotation tools, and safety frameworks. \n\nDario also advised me that when working with Anthropic, it's important to have a clear understanding of your use case and goals. Anthropic's team can then help you identify the right solutions and provide guidance on implementation.\n\nAdditionally, Dario mentioned that Anthropic places a strong emphasis on transparency and collaboration. They encourage users to engage with their research and provide feedback to help improve their offerings.\n\nOverall, Dario's advice was to approach Anthropic with a collaborati

In [6]:
full_chain.invoke({"question": "how do I use LangChain?"})

AIMessage(content='As Harrison Chase told me, LangChain is a framework for building applications with large language models (LLMs). It provides a set of abstractions and tools that make it easier to build applications using LLMs, such as chatbots, question-answering systems, and text generation applications.\n\nTo use LangChain, you can start by installing the library and importing the necessary components. For example, you can use the `OpenAILLMChain` to interact with the OpenAI API and generate text. Here\'s a basic example:\n\n```python\nfrom langchain.llms import OpenAI\nfrom langchain.chains import LLMChain\nfrom langchain.prompts import PromptTemplate\n\nllm = OpenAI(temperature=0.7)\nprompt = PromptTemplate(\n    input_variables=["topic"],\n    template="Write a short essay about {topic}.",\n)\nchain = LLMChain(llm=llm, prompt=prompt)\nresult = chain.run("the history of LangChain")\nprint(result)\n```\n\nIn this example, we create an instance of the `OpenAILLMChain`, which allow

In [7]:
full_chain.invoke({"question": "whats 2 + 2"})

AIMessage(content='4', additional_kwargs={}, response_metadata={'id': 'msg_01WbQUGXnrzWZcj6YzQYhxmA', 'model': 'claude-3-haiku-20240307', 'stop_reason': 'end_turn', 'stop_sequence': None, 'usage': {'input_tokens': 28, 'output_tokens': 5}}, id='run-ee4feff3-6d03-4811-81ac-fc16647a1099-0', usage_metadata={'input_tokens': 28, 'output_tokens': 5, 'total_tokens': 33})

## Using a RunnableBranch

A `RunnableBranch` is a special type of runnable that allows you to define a set of conditions and runnables to execute based on the input. It does **not** offer anything that you can't achieve in a custom function as described above, so we recommend using a custom function instead.

A `RunnableBranch` is initialized with a list of (condition, runnable) pairs and a default runnable. It selects which branch by passing each condition the input it's invoked with. It selects the first condition to evaluate to True, and runs the corresponding runnable to that condition with the input. 

If no provided conditions match, it runs the default runnable.

Here's an example of what it looks like in action:

In [8]:
from langchain_core.runnables import RunnableBranch

branch = RunnableBranch(
    (lambda x: "anthropic" in x["topic"].lower(), anthropic_chain),
    (lambda x: "langchain" in x["topic"].lower(), langchain_chain),
    general_chain,
)
full_chain = {"topic": chain, "question": lambda x: x["question"]} | branch
full_chain.invoke({"question": "how do I use Anthropic?"})

AIMessage(content="As Dario Amodei told me, to use Anthropic, you can start by exploring the resources and tools available on the Anthropic website. Anthropic offers a range of products and services, including language models, AI development platforms, and research collaborations. \n\nTo get started, I'd recommend familiarizing yourself with Anthropic's core technology and mission, which is centered around building safe and ethical AI systems. Anthropic has a strong focus on issues like AI alignment, transparency, and the responsible development of AI.\n\nDepending on your specific needs and use case, Anthropic may be able to provide you with access to their language models or assist you in developing your own AI applications. They also have a team of experienced researchers and engineers who can offer guidance and support.\n\nOverall, Anthropic is committed to advancing the field of AI in a way that benefits humanity. I'd encourage you to reach out to the Anthropic team to learn more 

In [9]:
full_chain.invoke({"question": "how do I use LangChain?"})

AIMessage(content='As Harrison Chase told me, LangChain is a powerful framework for building applications with large language models (LLMs). It provides a set of abstractions and tools to make it easier to build applications that utilize LLMs. The key components of LangChain include:\n\n1. Agents: These are autonomous agents that can interact with environments and make decisions based on their observations and goals.\n\n2. Chains: These are sequences of steps that can be executed to accomplish a task, such as retrieving information from a database and then using an LLM to generate a response.\n\n3. Prompts: LangChain provides a set of tools for creating and managing prompts, which are the input that is provided to an LLM.\n\n4. Memory: LangChain includes various types of memory that can be used to store and retrieve information as an agent interacts with its environment.\n\nTo get started with LangChain, Harrison recommended checking out the documentation and tutorials on the LangChain

In [10]:
full_chain.invoke({"question": "whats 2 + 2"})

AIMessage(content='4', additional_kwargs={}, response_metadata={'id': 'msg_01BHA9ifGzwum1ydKMML2A5P', 'model': 'claude-3-haiku-20240307', 'stop_reason': 'end_turn', 'stop_sequence': None, 'usage': {'input_tokens': 28, 'output_tokens': 5}}, id='run-d90e490b-abab-4058-911c-5ea22105cda3-0', usage_metadata={'input_tokens': 28, 'output_tokens': 5, 'total_tokens': 33})

## Routing by semantic similarity

One especially useful technique is to use embeddings to route a query to the most relevant prompt. Here's an example.

In [11]:
from langchain_community.utils.math import cosine_similarity
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_openai import OpenAIEmbeddings

physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise and easy to understand manner. \
When you don't know the answer to a question you admit that you don't know.

Here is a question:
{query}"""

math_template = """You are a very good mathematician. You are great at answering math questions. \
You are so good because you are able to break down hard problems into their component parts, \
answer the component parts, and then put them together to answer the broader question.

Here is a question:
{query}"""

embeddings = OpenAIEmbeddings()
prompt_templates = [physics_template, math_template]
prompt_embeddings = embeddings.embed_documents(prompt_templates)


def prompt_router(input):
    query_embedding = embeddings.embed_query(input["query"])
    similarity = cosine_similarity([query_embedding], prompt_embeddings)[0]
    most_similar = prompt_templates[similarity.argmax()]
    print("Using MATH" if most_similar == math_template else "Using PHYSICS")
    return PromptTemplate.from_template(most_similar)


chain = (
    {"query": RunnablePassthrough()}
    | RunnableLambda(prompt_router)
    | ChatAnthropic(model="claude-3-haiku-20240307")
    | StrOutputParser()
)

/Users/bagatur/langchain/.venv/lib/python3.11/site-packages/pydantic/_internal/_config.py:341: UserWarning: Valid config keys have changed in V2:
* 'allow_population_by_field_name' has been renamed to 'populate_by_name'
  warnings.warn(message, UserWarning)


PydanticUserError: The `__modify_schema__` method is not supported in Pydantic v2. Use `__get_pydantic_json_schema__` instead in class `SecretStr`.

For further information visit https://errors.pydantic.dev/2.9/u/custom-json-schema

In [12]:
print(chain.invoke("What's a black hole"))

Other


In [13]:
print(chain.invoke("What's a path integral"))

Other


## Next steps

You've now learned how to add routing to your composed LCEL chains.

Next, check out the other how-to guides on runnables in this section.